# it's time to try defining entropy


1. function to define ROIs (one per "square")
2. create csv with sequence of rois visitation + time spent per visit 
3. function for $- log(p)$
4. function for spatial entropy 

$$H = - \sum p_i log(p_i)$$

- where probability will be calculated by time spent in roi, andanother calculation with probability per visitation count. 


In [ ]:
def get_rois(video):
    get_first_frame_of_video
    draw_rois
    save_coordinates_in_csv

return rois_csv

def get_trajectory_per_trial(trajectory_csv, )


